In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
%%time
item_df = pd.read_csv('item_dict.csv', delimiter = ',')
item_dict = dict(zip(item_df.en_label.values, [np.fromstring(i, dtype=int, sep=',') for i in item_df.item_id.values]))
del item_df
print(item_dict['tesla'])

FileNotFoundError: [Errno 2] File b'item_dict.csv' does not exist: b'item_dict.csv'

In [3]:
print(len(item_dict))

NameError: name 'item_dict' is not defined

In [ ]:
%%time
#unidirection will take ~2.5 min
#bidirectional will take ~5 min
graph_df = pd.read_csv('statements_uni.csv', delimiter = ',')
graph_dict = dict(zip(graph_df.Source.values, [np.fromstring(i, dtype=int, sep=',') for i in graph_df.Targets.values]))
del graph_df
print(graph_dict[9036])

In [ ]:
def traverse(root, levels=1):
    level = [np.unique(graph_dict.get(root))]
    
    for i in range(1, levels):
        temp = np.unique(np.concatenate([graph_dict.get(j, np.array([], dtype=int)) for j in level[i-1]]))
        level.append(np.setdiff1d(temp, np.concatenate(level)))
    
    return level

#for i in range(1, 11): 
#    print(f'{i} levels')
#    %time test = traverse(9036, i)
#    print(len(test[-1]), 'nodes')

In [ ]:
def Traverse(root, levels=1):
    
    if (type(graph_dict.get(root))==None):
        a =[]
        return a
    level = [np.unique(graph_dict.get(root))]
    #print (type(level))
    for i in range(1, levels):
        try:
            temp = np.unique(np.concatenate([graph_dict.get(j, np.array([], dtype=int)) for j in level[i-1]]))
        except:
            return []
        level.append(np.setdiff1d(temp, np.concatenate(level)))
    for i in range(len(level)):
        level[i]=set(level[i])
    return level



In [ ]:
a=Traverse(9036,10)

In [4]:
len(a)

NameError: name 'a' is not defined

In [9]:
    
def find_shortest_path_length(src,dst,maxlev,Dict):
    if dst==-1 or src ==-1:
        return 0
    if (src,dst) in Dict: return Dict[(src,dst)]
    else:
    
        pathlength=0
        found=False
        reachend=False
        level_sets=Traverse(src,maxlev)
        if level_sets==[]:
            Dict[(src,dst)]=5
            return 5
        level=0
    
        while ((not found) and (not reachend)):
            if (level==maxlev):
                reachend=True
        
            elif dst in level_sets[level]:
                pathlength=level
                found =True
            else:
                level+=1
        if found == True:
            Dict[(src,dst)]=level+1
            return level+1
        else:
            Dict[(src,dst)]=5
            return 5
        

In [10]:
%%time
find_shortest_path_length(9036,5507494,10, {})

Wall time: 4.11 s


10

In [11]:
find_shortest_path_length(9036,-1,10,{})

0

In [12]:
sentences = pickle.load(open("sentences.p", "rb"))

In [13]:
sentences[0]

([[275102],
  [1675432, 56538769, 30272204],
  [18366588, 217, 209754],
  [],
  [],
  [7913, 2175775, 57477615],
  [3627558, 2497107],
  [13211, 26245],
  [4135847, 4135849, 4135859, 7128194, 16562681, 17536409, 31283669, 11692785],
  [809, 385059, 22061936, 36982750, 57477613],
  []],
 array(['voivode', 'iuga', 'moldavia', '1400)(known', 'iurg', 'romanian',
        'yury', 'ruthenian', 'jerzy', 'polish', 'ologul'], dtype=object),
 ('Voivode', 149044, 275102))

In [1]:
len(sentences)

NameError: name 'sentences' is not defined

In [16]:
sentences[4][0]

[[48797348, 65807361, 35622],
 [776675, 48798668, 73002, 1343033, 27939009, 27939181, 30293377]]

In [17]:
sentences[4][2]

('SmackDown', 55778545, 48797348)

In [14]:
#pairwise for all => takes extremely long
def disambiguate(candidate_lists):
    Dict={}
    for i in range(len(candidate_lists)):
        if len(candidate_lists[i])==0:
            candidate_lists[i].append(-1)
    #print (candidate_lists)
    combination_list = list(itertools.product(*candidate_lists))
    #print(len(combination_list))
    disambiguation = combination_list[0]
    least = math.inf
    for combinations in combination_list:
        #print("dict is currently:")
        #print(Dict)
        distance_sum = 0
        #generate a list of pairs of candidates to calculate pairwise distances
        #print("current combination:", combinations)
        pairs = list(itertools.combinations(combinations,2))
        #print(pairs)
        #print(len(pairs))
        for pair in pairs:
            #print("current pair:", pair)
            pair_dist = find_shortest_path_length(pair[0],pair[1],3,Dict)
            #print("pair dist", pair_dist)
            distance_sum += pair_dist
        if (distance_sum < least):
            least = distance_sum
            disambiguation = combinations
        #print(least)
        #print("end of current combination==========")
    return disambiguation

In [34]:
sentences[4][2]

('SmackDown', 55778545, 48797348)

In [15]:
%%time
import itertools
import math
disambiguate(sentences[4][0])

Wall time: 95.8 ms


(65807361, 73002)

In [24]:
def total_count(nested_list):
    count =0
    for lists in nested_list:
        count = count + len(lists)
    return count

def prune_sentences(sentences):
    for sentence in sentences[:]:
        #print(sentence[0])
        sentence_length = len(sentence[0])
        total_disamb_count = total_count(sentence[0])
        if (len(sentence[0][0])==1 or sentence_length > 10 or total_disamb_count > 50):
            sentences.remove(sentence)

def prune_sentences2(sentences):
    for sentence in sentences[:]:
        #print(sentence[0])
        sentence_length = len(sentence[0])
        total_disamb_count = total_count(sentence[0])
        if (sentence_length > 10 or total_disamb_count > 100):
            sentences.remove(sentence)            

In [26]:
copy = list(sentences)
prune_sentences2(copy)

In [27]:
len(copy)

85491

## Disambiguation

In [ ]:
%%time
import itertools
import math

def disamb_sentences(sentences):
    cor_count=0
    count=0
    for sentence in sentences:
        count=count+1
        if (disambiguate(sentence[0])[0]==sentence[2][2]):
            cor_count = cor_count+1
        print(str(count/len(sentences))+"pecent done")
    return cor_count/len(sentences)

disamb_sentences(copy)

1.1697137710402265e-05pecent done
2.339427542080453e-05pecent done
3.5091413131206796e-05pecent done
4.678855084160906e-05pecent done
5.848568855201132e-05pecent done
7.018282626241359e-05pecent done
8.187996397281586e-05pecent done
9.357710168321812e-05pecent done
0.00010527423939362039pecent done
0.00011697137710402264pecent done
0.00012866851481442492pecent done
0.00014036565252482718pecent done
0.00015206279023522945pecent done
0.00016375992794563171pecent done
0.00017545706565603398pecent done
0.00018715420336643624pecent done
0.0001988513410768385pecent done
0.00021054847878724077pecent done
0.00022224561649764304pecent done
0.00023394275420804528pecent done
0.00024563989191844754pecent done
0.00025733702962884984pecent done
0.0002690341673392521pecent done
0.00028073130504965437pecent done
0.0002924284427600566pecent done
0.0003041255804704589pecent done
0.00031582271818086113pecent done
0.00032751985589126343pecent done
0.00033921699360166567pecent done
0.00035091413131206796pe

0.0029827701161525776pecent done
0.00299446725386298pecent done
0.003006164391573382pecent done
0.003017861529283784pecent done
0.0030295586669941863pecent done
0.0030412558047045886pecent done
0.003052952942414991pecent done
0.0030646500801253932pecent done
0.0030763472178357955pecent done
0.003088044355546198pecent done
0.0030997414932566pecent done
0.0031114386309670024pecent done
0.0031231357686774047pecent done
0.003134832906387807pecent done
0.0031465300440982093pecent done
0.0031582271818086116pecent done
0.003169924319519014pecent done
0.003181621457229416pecent done
0.0031933185949398184pecent done
0.0032050157326502203pecent done
0.0032167128703606226pecent done
0.003228410008071025pecent done
0.003240107145781427pecent done
0.0032518042834918295pecent done
0.0032635014212022318pecent done
0.003275198558912634pecent done
0.0032868956966230363pecent done
0.0032985928343334386pecent done
0.003310289972043841pecent done
0.0033219871097542432pecent done
0.0033336842474646455pecen

0.006000631645436362pecent done
0.006012328783146764pecent done
0.006024025920857166pecent done
0.006035723058567568pecent done
0.006047420196277971pecent done
0.006059117333988373pecent done
0.006070814471698775pecent done
0.006082511609409177pecent done
0.00609420874711958pecent done
0.006105905884829982pecent done
0.006117603022540385pecent done
0.0061293001602507865pecent done
0.006140997297961189pecent done
0.006152694435671591pecent done
0.006164391573381994pecent done
0.006176088711092396pecent done
0.006187785848802798pecent done
0.0061994829865132pecent done
0.006211180124223602pecent done
0.006222877261934005pecent done
0.006234574399644407pecent done
0.006246271537354809pecent done
0.006257968675065211pecent done
0.006269665812775614pecent done
0.006281362950486016pecent done
0.0062930600881964185pecent done
0.00630475722590682pecent done
0.006316454363617223pecent done
0.006328151501327625pecent done
0.006339848639038028pecent done
0.00635154577674843pecent done
0.006363242

0.009018493174720145pecent done
0.009030190312430549pecent done
0.00904188745014095pecent done
0.009053584587851353pecent done
0.009065281725561754pecent done
0.009076978863272158pecent done
0.00908867600098256pecent done
0.009100373138692962pecent done
0.009112070276403364pecent done
0.009123767414113767pecent done
0.009135464551824169pecent done
0.009147161689534571pecent done
0.009158858827244973pecent done
0.009170555964955375pecent done
0.009182253102665778pecent done
0.00919395024037618pecent done
0.009205647378086582pecent done
0.009217344515796984pecent done
0.009229041653507387pecent done
0.00924073879121779pecent done
0.009252435928928191pecent done
0.009264133066638593pecent done
0.009275830204348997pecent done
0.009287527342059398pecent done
0.0092992244797698pecent done
0.009310921617480202pecent done
0.009322618755190604pecent done
0.009334315892901008pecent done
0.00934601303061141pecent done
0.009357710168321811pecent done
0.009369407306032213pecent done
0.0093811044437

0.01203635470400393pecent done
0.012048051841714333pecent done
0.012059748979424734pecent done
0.012071446117135136pecent done
0.01208314325484554pecent done
0.012094840392555942pecent done
0.012106537530266344pecent done
0.012118234667976745pecent done
0.012129931805687149pecent done
0.01214162894339755pecent done
0.012153326081107953pecent done
0.012165023218818355pecent done
0.012176720356528758pecent done
0.01218841749423916pecent done
0.012200114631949562pecent done
0.012211811769659964pecent done
0.012223508907370367pecent done
0.01223520604508077pecent done
0.012246903182791171pecent done
0.012258600320501573pecent done
0.012270297458211975pecent done
0.012281994595922378pecent done
0.01229369173363278pecent done
0.012305388871343182pecent done
0.012317086009053584pecent done
0.012328783146763988pecent done
0.01234048028447439pecent done
0.012352177422184791pecent done
0.012363874559895193pecent done
0.012375571697605597pecent done
0.012387268835315999pecent done
0.0123989659730

...

0.26217964464095633pecent done
0.26219134177866676pecent done
0.26220303891637714pecent done
0.26221473605408757pecent done
0.26222643319179795pecent done
0.2622381303295084pecent done
0.26224982746721875pecent done
0.2622615246049292pecent done
0.26227322174263956pecent done
0.26228491888035pecent done
0.26229661601806037pecent done
0.2623083131557708pecent done
0.2623200102934812pecent done
0.2623317074311916pecent done
0.262343404568902pecent done
0.2623551017066124pecent done
0.2623667988443228pecent done
0.2623784959820332pecent done
0.2623901931197436pecent done
0.262401890257454pecent done
0.2624135873951644pecent done
0.2624252845328748pecent done
0.2624369816705852pecent done
0.2624486788082956pecent done
0.262460375946006pecent done
0.2624720730837164pecent done
0.2624837702214268pecent done
0.2624954673591372pecent done
0.26250716449684763pecent done
0.262518861634558pecent done
0.26253055877226844pecent done
0.2625422559099788pecent done
0.26255395304768925pecent done
0.262

0.2653612660981858pecent done
0.26537296323589615pecent done
0.2653846603736066pecent done
0.26539635751131696pecent done
0.2654080546490274pecent done
0.26541975178673777pecent done
0.2654314489244482pecent done
0.2654431460621586pecent done
0.265454843199869pecent done
0.2654665403375794pecent done
0.2654782374752898pecent done
0.2654899346130002pecent done
0.2655016317507106pecent done
0.265513328888421pecent done
0.2655250260261314pecent done
0.2655367231638418pecent done
0.26554842030155223pecent done
0.2655601174392626pecent done
0.26557181457697304pecent done
0.2655835117146834pecent done
0.2655952088523938pecent done
0.2656069059901042pecent done
0.2656186031278146pecent done
0.26563030026552503pecent done
0.2656419974032354pecent done
0.26565369454094584pecent done
0.2656653916786562pecent done
0.26567708881636665pecent done
0.265688785954077pecent done
0.26570048309178745pecent done
0.26571218022949783pecent done
0.26572387736720826pecent done
0.26573557450491864pecent done
0

0.2685311904177048pecent done
0.26854288755541517pecent done
0.2685545846931256pecent done
0.268566281830836pecent done
0.2685779789685464pecent done
0.2685896761062568pecent done
0.2686013732439672pecent done
0.2686130703816776pecent done
0.268624767519388pecent done
0.2686364646570984pecent done
0.2686481617948088pecent done
0.2686598589325192pecent done
0.26867155607022963pecent done
0.26868325320794pecent done
0.26869495034565044pecent done
0.2687066474833608pecent done
0.26871834462107125pecent done
0.2687300417587816pecent done
0.26874173889649206pecent done
0.26875343603420243pecent done
0.2687651331719128pecent done
0.26877683030962324pecent done
0.2687885274473336pecent done
0.26880022458504405pecent done
0.2688119217227544pecent done
0.26882361886046485pecent done
0.26883531599817523pecent done
0.26884701313588566pecent done
0.26885871027359604pecent done
0.26887040741130647pecent done
0.26888210454901684pecent done
0.2688938016867273pecent done
0.26890549882443765pecent done

0.27168941759951337pecent done
0.2717011147372238pecent done
0.2717128118749342pecent done
0.2717245090126446pecent done
0.271736206150355pecent done
0.2717479032880654pecent done
0.2717596004257758pecent done
0.2717712975634862pecent done
0.2717829947011966pecent done
0.27179469183890703pecent done
0.2718063889766174pecent done
0.27181808611432784pecent done
0.2718297832520382pecent done
0.27184148038974865pecent done
0.271853177527459pecent done
0.27186487466516945pecent done
0.27187657180287983pecent done
0.27188826894059026pecent done
0.27189996607830064pecent done
0.27191166321601107pecent done
0.27192336035372144pecent done
0.2719350574914318pecent done
0.27194675462914225pecent done
0.2719584517668526pecent done
0.27197014890456306pecent done
0.27198184604227343pecent done
0.27199354317998387pecent done
0.27200524031769424pecent done
0.2720169374554047pecent done
0.27202863459311505pecent done
0.2720403317308255pecent done
0.27205202886853586pecent done
0.2720637260062463pecent 

0.274847644781322pecent done


In [23]:
%%time
import itertools
import math

def disamb_sentences(sentences):
    cor_count=0
    for sentence in sentences:
    
        if (disambiguate(sentence[0])[0]==sentence[2][2]):
            cor_count = cor_count+1
    return cor_count/len(sentences)

disamb_sentences(copy[0:10000])

Wall time: 1h 1min 26s


0.7452

https://arxiv.org/abs/1802.01074
http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.557.136
https://www.kaggle.com/gabrielaltay/kdwd-subclass-path-ner


In [29]:
%%time
import itertools
import math
from multiprocessing import Pool


def process_image(sentence):
    cor_count=0
    if (disambiguate(sentence[0])[0]==sentence[2][2]):
        cor_count = cor_count+1
    return cor_count

if __name__ == '__main__':
    pool = Pool(os.cpu_count()-1)
    pool.map(process_image, sentences)  
    
disambiguate(sentences[0][0])

[[275102], [1675432, 56538769, 30272204], [18366588, 217, 209754], [-1], [-1], [7913, 2175775, 57477615], [3627558, 2497107], [13211, 26245], [4135847, 4135849, 4135859, 7128194, 16562681, 17536409, 31283669, 11692785], [809, 385059, 22061936, 36982750, 57477613], [-1]]
4320
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<cla

(275102, 30272204, 217, -1, -1, 7913, 3627558, 13211, 17536409, 809, -1)